In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import math
import matplotlib.pyplot as plt
import torch.nn.init as init
import time
import random
import pandas as pd

In [12]:
def np_to_torch(arr):
    
    arr = torch.FloatTensor(arr)
    arr = arr.unsqueeze(-1)
    arr = arr.clone().detach().requires_grad_(True)
    
    return arr

def xyt_train_data(N_x, x_l, x_r, N_y, y_b, y_t, N_t, t_i, t_f, N_bc):

    start = []
    
    ####  Spatial array
    x_r_train = 1*x_r
    y_t_train = 1*y_t
    N_x_train = int(x_r_train/x_r*N_x) 
    N_y_train = int(y_t_train/y_t*N_y) 
    
    x_train = np.linspace(x_l, x_r_train, N_x_train)  
    y_train = np.linspace(y_b, y_t, N_y)
    x_train = np.repeat(x_train, N_y)
    y_train = np.tile(y_train, N_x_train)
    
    n = x_train.shape[0]
    x_train = np.tile(x_train, N_t)
    y_train = np.tile(y_train, N_t)
    t_train = np.linspace(t_i, t_f, N_t)
    t_train = np.repeat(t_train, n )
    
    start.append( 0 )
    start.append( t_train.shape[0] ) 
    print('n = ', n)
    print('training data-size = ', t_train.shape[0] )
    
    # Boundary collocation points
    x_bc1 = np.ones((N_bc))*x_l
    y_bc1 = np.linspace(y_b, y_t, N_bc)
    t_bc1 = np.linspace(t_i+0.005,t_f,N_t)
    x_bc1 = np.tile(x_bc1, N_t)
    y_bc1 = np.tile(y_bc1, N_t)
    t_bc1 = np.repeat(t_bc1, N_bc)
    
    start.append( start[-1] + t_bc1.shape[0] )
    
    x_bc2 = np.ones((N_bc))*x_r
    y_bc2 = np.linspace(y_b+0.001,y_t,N_bc)
    t_bc2 = np.linspace(t_i+0.005,t_f,N_t)
    x_bc2 = np.tile(x_bc2, N_t)
    y_bc2 = np.tile(y_bc2, N_t)
    t_bc2 = np.repeat(t_bc2, N_bc)
    
    start.append( start[-1] + t_bc2.shape[0] )
    
    x_bc3 = np.linspace(x_l,x_r,N_bc)
    y_bc3 = np.ones((N_bc))*y_b
    t_bc3 = np.linspace(t_i+0.005,t_f,N_t)
    x_bc3 = np.tile(x_bc3, N_t)
    y_bc3 = np.tile(y_bc3, N_t)
    t_bc3 = np.repeat(t_bc3, N_bc)
    
    start.append( start[-1] + t_bc3.shape[0] )
    
    x_bc4 = np.linspace(x_l+0.001,x_r,N_bc)
    y_bc4 = np.ones((N_bc))*y_t
    t_bc4 = np.linspace(t_i+0.005,t_f,N_t)
    x_bc4 = np.tile(x_bc4, N_t)
    y_bc4 = np.tile(y_bc4, N_t)
    t_bc4 = np.repeat(t_bc4, N_bc)
    
    start.append( start[-1] + t_bc4.shape[0] )
    
    # np to torch
    x_T = np.concatenate((x_train, x_bc1, x_bc2, x_bc3, x_bc4), axis = 0)
    y_T = np.concatenate((y_train, y_bc1, y_bc2, y_bc3, y_bc4), axis = 0)
    t_T = np.concatenate((t_train, t_bc1, t_bc2, t_bc3, t_bc4), axis = 0)
    x_T = np_to_torch(x_T)
    y_T = np_to_torch(y_T)
    t_T = np_to_torch(t_T)
    
    return x_T, y_T, t_T, start, n

class RBF(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, x):
        y = 1.2*torch.exp(-5*torch.square(x)) - 0.1
        return y

def xavier_init(m):
    if isinstance(m, nn.Linear):
        init.xavier_normal_(m.weight)
        init.xavier_normal_(m.bias)
    
class ANN(nn.Module):
    def __init__(self, layer_size_1):
        super(ANN, self).__init__()

        self.act = RBF()
        
        ########### Fully conected model-1 ###########
        modules_11 = []
        modules_11.append(nn.Linear(layer_size_1[0], layer_size_1[1]))  
        modules_11.append(self.act)
        modules_11.append(nn.Linear(layer_size_1[1], layer_size_1[2]))  
        self.fc_11 = nn.Sequential( *modules_11 )
                
        modules_12 = []
        modules_12.append(nn.Linear(layer_size_1[1], layer_size_1[2])) 
        modules_12.append(self.act)
        modules_12.append(nn.Linear(layer_size_1[1], layer_size_1[2])) 
        modules_12.append(self.act)
        self.fc_12 = nn.Sequential( *modules_12 )
        
        modules_13 = []
        modules_13.append(nn.Linear(layer_size_1[2], layer_size_1[3])) 
        self.fc_13 = nn.Sequential( *modules_13 )

        for layer in [self.fc_11.modules(), self.fc_12.modules(), self.fc_13.modules()]:
            if isinstance(layer, nn.Linear):
                xavier_init(layer)

    def forward(self, x_T, y_T, t_T, start, result, pr):

        ########### Fully conected model-1 ###########
        out_1 = self.fc_11( torch.cat((x_T, y_T, t_T),1) )
        out_2 = self.fc_12( out_1 )
        T = self.fc_13( out_1 + out_2 )
        
        dTdx = torch.autograd.grad(T, x_T, grad_outputs=torch.ones_like(T), create_graph=True)[0]
        d2Tdx2 = torch.autograd.grad(dTdx, x_T, grad_outputs=torch.ones_like(dTdx), create_graph=True)[0]
        dTdy = torch.autograd.grad(T, y_T, grad_outputs=torch.ones_like(T), create_graph=True)[0]
        d2Tdy2 = torch.autograd.grad(dTdy, y_T, grad_outputs=torch.ones_like(dTdy), create_graph=True)[0]
        dTdt = torch.autograd.grad(T, t_T, grad_outputs=torch.ones_like(T), create_graph=True)[0]

        return  T, dTdx, dTdy, d2Tdx2, d2Tdy2, dTdt
    
def get_loss(x_T, y_T, t_T, start, k1, k2, w1, w2, w3, w4, w5, w6, w7, q_norm, n, T_change, N_t):
 
    T, dTdx, dTdy, d2Tdx2, d2Tdy2, dTdt = model( x_T, y_T, t_T, start, 0, 0)

    eq1 = w1*( torch.sum( torch.mul( torch.square( dTdt - k1*(d2Tdx2 + d2Tdy2)), torch.where(T>=T_change,1,0) ) ) 
    + torch.sum( torch.mul( torch.square( dTdt - k2*(d2Tdx2 + d2Tdy2)), torch.where(T<T_change,1,0) ) ) )/(n*N_t)
    bc1 = w2*torch.sum( torch.square( dTdx[ start[1]:start[2] ] + q_norm ) )/(start[2]-start[1]) #left boundary
    bc2 = w3*torch.sum( torch.square( dTdy[ start[3]:start[4] ] + q_norm  ) )/(start[4]-start[3]) #bottom boundary
    bc3 = w4*torch.sum( torch.square( dTdy[ start[4]:start[5] ] ) )/(start[5]-start[4]) #top boundary
    bc4 = w5*torch.sum( torch.square( dTdx[ start[2]:start[3] ] ) )/(start[3]-start[2]) #right boundary
    ic1 = w6*torch.sum( torch.square( T[0:n] )  )/(n) 
    
    loss = eq1 + bc1 + bc2 + bc3 + bc4 + ic1 
    
    return loss, eq1, bc1, bc2, bc3, bc4, ic1

def print_loss(epoch, loss, eq1, bc1, bc2, bc3, bc4, ic1):
    print('epoch = ',epoch)
    print('loss = ',loss.detach().numpy())
    print('eq1_loss = ',eq1.detach().numpy())
    print('bc1_loss = ',bc1.detach().numpy())
    print('bc2_loss = ',bc2.detach().numpy())
    print('bc3_loss = ',bc3.detach().numpy())
    print('bc4_loss = ',bc4.detach().numpy())
    print('ic1_loss = ',ic1.detach().numpy())

def save_model(model, path):
    torch.save(model.state_dict(), path)

def load_model(model, path):
    model.load_state_dict(torch.load(path))
    model.eval()

In [ ]:
#### material params:- RT-35 ####
k_therm = 0.2
L = 160000
Cp = 2000
rho = 800 
T_solidus = 305
T_liquidus = 311

#### Heat flux ####
q = 225

#### Normalising coeffs ####
T_change = 0.35
delta_x = 0.02
delta_y = 0.02
delta_T = 17.14
delta_t = 2400

#### Normalised Coefficients  ####
k1 = k_therm/(rho*Cp*delta_x**2)*delta_t
k2 = k_therm/(rho*(Cp+L/(T_liquidus - T_solidus))*delta_x**2)*delta_t
q_norm = q*delta_x/(k_therm*delta_T)

#### Neural Network Parameters ####
layer_size_1 = [3, 20, 20, 1]
model = ANN(layer_size_1)

#### optimizer and scheduler ####
lr1 = 1e-3
optimiser1 = torch.optim.Rprop(model.parameters(), lr=lr1)
epochs = 10000
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimiser1, T_max=epochs)

#### Collocation points ####
x_l = 0
y_b = 0
x_r = 1
y_t = 1
t_i = 0
t_f = 2.4
N_x = 66
N_y = 66
N_t = 200
N_bc = 66
x_T, y_T, t_T, start, n = xyt_train_data(N_x, x_l, x_r, N_y, y_b, y_t, N_t, t_i, t_f, N_bc)

In [ ]:
# Load model
model.train()  

# Loss function weights
w1 = 1
w2 = 1
w3 = 1
w4 = 1
w5 = 1
w6 = 50
w7 = 1

for epoch in range(epochs):        
    #Backpropogation and optimisation
    w7 = 1
    loss, eq1, bc1, bc2, bc3, bc4, ic1 =  get_loss(x_T, y_T, t_T, start, k1, k2, w1, w2, w3, w4, w5, w6, w7, q_norm, n, T_change, N_t)
    optimiser1.zero_grad()
    loss.backward()
    optimiser1.step()
    scheduler.step()

    if epoch%1==0:
        print_loss(epoch, loss, eq1, bc1, bc2, bc3, bc4, ic1)
        print("")